In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv
/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv


In [73]:
import re
import numpy as np
import pandas as pd
import emoji
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional, SimpleRNN, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt')
nltk.download('punkt_tab') 

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
train_df=pd.read_csv("/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv")
train_df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [7]:
val_df=pd.read_csv("/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv")
val_df.head()

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [8]:
val_df=val_df.rename(columns={
    "3364":"ID",
    "Facebook":"Entity",
    "Irrelevant":"Label",
   "I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣":"Text"
})

In [10]:
train_df=train_df.rename(columns={
    "2401":'ID',
    "Borderlands":"Entity",
    "Positive":"Label",
    "im getting on borderlands and i will murder you all ,":"Text" 
})

In [12]:
train_df= train_df.drop_duplicates()
val_df= val_df.drop_duplicates()

In [13]:
train_df["Text"].isnull().sum()
train_df=train_df.dropna()

In [41]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [74]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'@\w+', '', text)           # remove mentions
    text = re.sub(r'#\w+', '', text)           # remove hashtags
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # remove URLs
    text = emoji.replace_emoji(text, replace='') 
    text = re.sub(r'[^\w\s]', '', text)        # remove punctuation
    text = re.sub(r'\d+', '', text)            # remove digits
    text = re.sub(r'\s+', ' ', text).strip()   # remove extra spaces
    return text

In [43]:
def lemmatize_text(text):
    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return ' '.join(words)

In [16]:
train_df["Text"]=train_df["Text"].apply(clean_text)
val_df["Text"]=val_df["Text"].apply(clean_text)

In [19]:
train_df["Text"] = train_df["Text"].apply(lemmatize_text)
val_df["Text"] = val_df["Text"].apply(lemmatize_text)


In [36]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [44]:
x_train = train_df["Text"].tolist()
y_train = train_df["Label"].tolist()
x_test = val_df["Text"].tolist()
y_test = val_df["Label"].tolist()


In [45]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [46]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [47]:
def tokenize_and_pad(x_train, x_test, vocab_size=5000, max_len=120):
    tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
    tokenizer.fit_on_texts(x_train)

    # Convert texts to sequences
    X_train_seq = tokenizer.texts_to_sequences(x_train)
    X_test_seq = tokenizer.texts_to_sequences(x_test)

    # Pad sequences
    X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
    X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

    return X_train_pad, X_test_pad, tokenizer

In [48]:
x_train = train_df["Text"].astype(str).tolist()
x_test = val_df["Text"].astype(str).tolist()


In [98]:
val_df_test=val_df.drop(["ID","Entity","Label"],axis=1)

In [100]:
val_texts = val_df_test["Text"].values 
val_seq = tokenizer.texts_to_sequences(val_texts)
val_pad = pad_sequences(val_seq, maxlen=max_len, padding='post', truncating='post')

In [49]:
X_train_pad, X_test_pad, tokenizer = tokenize_and_pad(x_train, x_test, vocab_size=5000, max_len=120)
print(X_train_pad.shape)
print(X_test_pad.shape)

(71655, 120)
(999, 120)


In [75]:
class_weights = compute_class_weight(
    'balanced', classes=np.unique(y_train), y=y_train
)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)


Class Weights: {0: 1.4288705431921513, 1: 0.8255945248409992, 2: 1.0116190422407951, 3: 0.9087738433441559}


In [78]:
vocab_size = 5000
max_len = 120
epochs = 20
batch_size = 64


In [77]:
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True,
    mode='max'
)

In [79]:
def build_deep_rnn(vocab_size, max_len):
    model = Sequential([
        Embedding(vocab_size, 128, input_length=max_len),
        Bidirectional(SimpleRNN(128, return_sequences=True)),
        BatchNormalization(),
        Dropout(0.5),
        Bidirectional(SimpleRNN(64)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(4, activation='softmax')  # 4-class output
    ])
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(learning_rate=0.0005),
        metrics=['accuracy']
    )
    return model

In [80]:
rnn_model = build_deep_rnn(vocab_size, max_len)
rnn_model.build(input_shape=(None, max_len))
rnn_model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ (None, 120, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 120, 256)       │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 120, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 120, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 748,932 (2.86 MB)

 Trainable params: 748,164 (2.85 MB)

 Non-trainable params: 768 (3.00 KB)

In [83]:
history_rnn = rnn_model.fit(
    X_train_pad, y_train,
    validation_data=(X_test_pad, y_test),
    epochs=20,
    batch_size=128,
    class_weight=class_weights,
    callbacks=[early_stop]
)

Epoch 1/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 41s 54ms/step - accuracy: 0.3104 - loss: 1.8881 - val_accuracy: 0.2803 - val_loss: 2.2122
Epoch 2/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 45ms/step - accuracy: 0.6220 - loss: 0.9476 - val_accuracy: 0.4995 - val_loss: 1.4278
Epoch 3/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 45ms/step - accuracy: 0.7416 - loss: 0.6811 - val_accuracy: 0.5105 - val_loss: 1.4359
Epoch 4/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.8053 - loss: 0.5229 - val_accuracy: 0.6977 - val_loss: 1.1021
Epoch 5/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.8413 - loss: 0.4349 - val_accuracy: 0.8619 - val_loss: 0.4125
Epoch 6/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.8661 - loss: 0.3647 - val_accuracy: 0.3564 - val_loss: 3.0229
Epoch 7/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.8781 - loss: 0.3320 - val_accuracy: 0.7467 - val_loss: 1.0037
Epoch 8/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.8880 - loss: 0.3027 - 

In [88]:
def build_deep_lstm(vocab_size, max_len):
    model = Sequential([
        Embedding(vocab_size, 128, input_length=max_len),
        Bidirectional(LSTM(128, return_sequences=True)),
        BatchNormalization(),
        Dropout(0.5),
        Bidirectional(LSTM(64)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(4, activation='softmax')  # 4-class classification
    ])
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(learning_rate=0.0005),
        metrics=['accuracy']
    )
    return model


In [89]:
lstm_model = build_deep_lstm(vocab_size, max_len)
lstm_model.build(input_shape=(None, max_len))
lstm_model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)        │ (None, 120, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 120, 256)       │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 120, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 120, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,069,572 (4.08 MB)

 Trainable params: 1,068,804 (4.08 MB)

 Non-trainable params: 768 (3.00 KB)

In [90]:
history_lstm = lstm_model.fit(
   X_train_pad, y_train,
    validation_data=(X_test_pad, y_test),
    epochs=20,
    batch_size=128,
    class_weight=class_weights,
    callbacks=[early_stop]
)

Epoch 1/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 34s 46ms/step - accuracy: 0.4258 - loss: 1.5602 - val_accuracy: 0.6807 - val_loss: 0.7748
Epoch 2/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.7138 - loss: 0.7373 - val_accuracy: 0.7768 - val_loss: 0.7590
Epoch 3/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.7821 - loss: 0.5761 - val_accuracy: 0.7057 - val_loss: 1.0194
Epoch 4/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.8189 - loss: 0.4783 - val_accuracy: 0.9029 - val_loss: 0.2894
Epoch 5/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.8449 - loss: 0.4107 - val_accuracy: 0.8819 - val_loss: 0.3267
Epoch 6/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.8633 - loss: 0.3599 - val_accuracy: 0.9249 - val_loss: 0.2449
Epoch 7/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.8757 - loss: 0.3264 - val_accuracy: 0.9049 - val_loss: 0.2948
Epoch 8/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.8893 - loss: 0.2916 - 

In [94]:
def build_deep_gru(vocab_size, max_len):
    model = Sequential([
        Embedding(vocab_size, 128, input_length=max_len),
        Bidirectional(GRU(128, return_sequences=True)),
        BatchNormalization(),
        Dropout(0.5),
        Bidirectional(GRU(64)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(4, activation='softmax')  # 4-class classification
    ])
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(learning_rate=0.0005),
        metrics=['accuracy']
    )
    return model

In [95]:
gru_model = build_deep_gru(vocab_size, max_len)
gru_model.build(input_shape=(None, max_len))
gru_model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_13 (Embedding)        │ (None, 120, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_8 (Bidirectional) │ (None, 120, 256)       │       198,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 120, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 120, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ (None, 128)            │       123,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 963,844 (3.68 MB)

 Trainable params: 963,076 (3.67 MB)

 Non-trainable params: 768 (3.00 KB)

In [93]:
history_gru = gru_model.fit(
 X_train_pad, y_train,
    validation_data=(X_test_pad, y_test),
    epochs=20,
    batch_size=128,
    class_weight=class_weights,
    callbacks=[early_stop]
)


Epoch 1/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 31s 46ms/step - accuracy: 0.3913 - loss: 1.6216 - val_accuracy: 0.7017 - val_loss: 0.7663
Epoch 2/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.6748 - loss: 0.8179 - val_accuracy: 0.8018 - val_loss: 0.5297
Epoch 3/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 45ms/step - accuracy: 0.7552 - loss: 0.6378 - val_accuracy: 0.8498 - val_loss: 0.4444
Epoch 4/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.8031 - loss: 0.5268 - val_accuracy: 0.8579 - val_loss: 0.4361
Epoch 5/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.8346 - loss: 0.4413 - val_accuracy: 0.8078 - val_loss: 0.5511
Epoch 6/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.8571 - loss: 0.3815 - val_accuracy: 0.8649 - val_loss: 0.4198
Epoch 7/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 45ms/step - accuracy: 0.8745 - loss: 0.3364 - val_accuracy: 0.8519 - val_loss: 0.4374
Epoch 8/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 25s 45ms/step - accuracy: 0.8890 - loss: 0.3027 - 

In [103]:
val_predictions = gru_model.predict(val_pad)

32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step


In [104]:
predicted_classes = np.argmax(val_predictions, axis=1)  # For multi-class classification

print(predicted_classes)

[2 2 2 2 2 3 2 2 2 2 0 2 2 2 2 2 2 0 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0
 2 2 2 0 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 3 0 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 0 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 2 0 2 2 2 2 2 2 2 2 0 2 2
 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 0 2 2 3 2 2 2 2 2 2 2 2 2
 2 2 2 0 2 2 2 2 2 3 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 0 2
 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 0 2 2 2 2 2 2 2 2 0 3 2 2 2 2 0 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2
 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 0 2 0 2 2 2 2 2 2 2 0 2 0 2 2 2 2 2 0 2 2 2
 2 3 2 2 2 0 2 2 2 0 2 2 2 2 2 0 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2
 2 2 0 2 2 2 0 2 0 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2
 2 2 2 0 2 2 2 2 2 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2
 0 2 2 2 2 0 2 2 2 2 0 2 